#### Import Libraries

In [47]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [ ]:
from pyspark.sql import types
from pyspark.sql.functions import col
from pyspark.sql import types
from pyspark.sql.functions import col
from pyspark import SparkConf, SparkContext

In [49]:
import requests, pandas as pd, numpy as np
from pandas import DataFrame
from io import StringIO
import time, json
from datetime import date
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error
import matplotlib.pylab as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

#### Create Spark session

In [50]:
# Reason why we have the getOrCreate code
# http://stackoverflow.com/questions/28999332/how-to-access-sparkcontext-in-pyspark-script
sc = SparkContext.getOrCreate()

#### Load Data (Apple Stock price data)

In [51]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Stock_price_prediction').getOrCreate()
df = spark.read.csv(r"D:/MATH 5671/Group 7/Project/AAPL.csv", header = True, inferSchema = True)
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj_Close: string (nullable = true)
 |-- Volume: string (nullable = true)



In [52]:
#displaying the data
df.show()

+-------------------+--------+--------+--------+--------+---------+---------+
|               Date|    Open|    High|     Low|   Close|Adj_Close|   Volume|
+-------------------+--------+--------+--------+--------+---------+---------+
|1980-12-12 00:00:00|0.513393|0.515625|0.513393|0.513393| 0.023007|117258400|
|1980-12-15 00:00:00|0.488839|0.488839|0.486607|0.486607| 0.021807| 43971200|
|1980-12-16 00:00:00|0.453125|0.453125|0.450893|0.450893| 0.020206| 26432000|
|1980-12-17 00:00:00|0.462054|0.464286|0.462054|0.462054| 0.020706| 21610400|
|1980-12-18 00:00:00|0.475446|0.477679|0.475446|0.475446| 0.021307| 18362400|
|1980-12-19 00:00:00|0.504464|0.506696|0.504464|0.504464| 0.022607| 12157600|
|1980-12-22 00:00:00|0.529018| 0.53125|0.529018|0.529018| 0.023707|  9340800|
|1980-12-23 00:00:00|0.551339|0.553571|0.551339|0.551339| 0.024708| 11737600|
|1980-12-24 00:00:00|0.580357|0.582589|0.580357|0.580357| 0.026008| 12000800|
|1980-12-26 00:00:00|0.633929|0.636161|0.633929|0.633929| 0.0284

In [53]:
#shape of dataframe
len(df.columns), df.count()

7

#### Feature Engineering

In [55]:
# creating features to show daily variance of price
df = df.withColumn('O-L', df['Open'] - df['Low'])
df = df.withColumn('O-C', df.Open - df.Adj_Close)
df = df.withColumn('H-L', df.High - df.Low)
df.show()

+-------------------+--------+--------+--------+--------+---------+---------+--------------------+
|               Date|    Open|    High|     Low|   Close|Adj_Close|   Volume|                 O-L|
+-------------------+--------+--------+--------+--------+---------+---------+--------------------+
|1980-12-12 00:00:00|0.513393|0.515625|0.513393|0.513393| 0.023007|117258400|                 0.0|
|1980-12-15 00:00:00|0.488839|0.488839|0.486607|0.486607| 0.021807| 43971200|0.002232000000000...|
|1980-12-16 00:00:00|0.453125|0.453125|0.450893|0.450893| 0.020206| 26432000|0.002232000000000...|
|1980-12-17 00:00:00|0.462054|0.464286|0.462054|0.462054| 0.020706| 21610400|                 0.0|
|1980-12-18 00:00:00|0.475446|0.477679|0.475446|0.475446| 0.021307| 18362400|                 0.0|
|1980-12-19 00:00:00|0.504464|0.506696|0.504464|0.504464| 0.022607| 12157600|                 0.0|
|1980-12-22 00:00:00|0.529018| 0.53125|0.529018|0.529018| 0.023707|  9340800|                 0.0|
|1980-12-2

In [56]:
from pyspark.sql.functions import lag, lit, lead
from pyspark.sql.window import Window

In [57]:
df = df.withColumn('dummy_column', lit('Stocks!'))

In [58]:
df.show()

+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+
|               Date|    Open|    High|     Low|   Close|Adj_Close|   Volume|                 O-L|dummy_column|
+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+
|1980-12-12 00:00:00|0.513393|0.515625|0.513393|0.513393| 0.023007|117258400|                 0.0|     Stocks!|
|1980-12-15 00:00:00|0.488839|0.488839|0.486607|0.486607| 0.021807| 43971200|0.002232000000000...|     Stocks!|
|1980-12-16 00:00:00|0.453125|0.453125|0.450893|0.450893| 0.020206| 26432000|0.002232000000000...|     Stocks!|
|1980-12-17 00:00:00|0.462054|0.464286|0.462054|0.462054| 0.020706| 21610400|                 0.0|     Stocks!|
|1980-12-18 00:00:00|0.475446|0.477679|0.475446|0.475446| 0.021307| 18362400|                 0.0|     Stocks!|
|1980-12-19 00:00:00|0.504464|0.506696|0.504464|0.504464| 0.022607| 12157600|                 0.0|     S

In [59]:
# Creating target column for regression
w = Window.orderBy("dummy_column")
value_lead = lead('Adj_Close', count=1).over(w)
df = df.withColumn('Next_Adj_Close', value_lead)

In [60]:
df.show(2)

+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+
|               Date|    Open|    High|     Low|   Close|Adj_Close|   Volume|                 O-L|dummy_column|Next_Adj_Close|
+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+
|1980-12-12 00:00:00|0.513393|0.515625|0.513393|0.513393| 0.023007|117258400|                 0.0|     Stocks!|      0.024508|
|1980-12-15 00:00:00|0.488839|0.488839|0.486607|0.486607| 0.021807| 43971200|0.002232000000000...|     Stocks!|      0.025008|
+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+
only showing top 2 rows



#### Creating features to represent seasonality in Data

In [65]:
from pyspark.sql.functions import to_date, date_format
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofmonth, weekofyear, dayofweek

In [68]:
# All the below features show seasonality of the data
df = df.withColumn('DATEFORMAT', to_date('Date'))
df = df.withColumn('YEAR', year('DATEFORMAT'))
df = df.withColumn('MONTH', month('DATEFORMAT'))
df = df.withColumn('DAYOFMONTH', dayofmonth('DATEFORMAT'))
df = df.withColumn('DAYOFMONTH', dayofmonth('DATEFORMAT'))
df = df.withColumn('DAYOFWEEK', dayofweek('DATEFORMAT'))
df = df.withColumn('WEEKOFYEAR', weekofyear('DATEFORMAT'))
df = df.withColumn("week", date_format('DATEFORMAT', "W"))

In [70]:
from pyspark.sql.functions import lag
from pyspark.sql.window import Window

In [71]:
w = Window().orderBy(df['DATE'])
df.show(2)

+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+
|               Date|    Open|    High|     Low|   Close|Adj_Close|   Volume|                 O-L|dummy_column|Next_Adj_Close|DATEFORMAT|YEAR|MONTH|DAYOFMONTH|DAYOFWEEK|WEEKOFYEAR|week|
+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+
|1980-12-12 00:00:00|0.513393|0.515625|0.513393|0.513393| 0.023007|117258400|                 0.0|     Stocks!|      0.024508|1980-12-12|1980|   12|        12|        6|        50|   2|
|1980-12-15 00:00:00|0.488839|0.488839|0.486607|0.486607| 0.021807| 43971200|0.002232000000000...|     Stocks!|      0.025008|1980-12-15|1980|   12|        15|        2|        51|   3|
+-------------------+--------+--------+--------+--------+---------+---

#### Creating weighted average features for previous periods to capture variability of data

In [72]:
from pyspark.sql.functions import coalesce, lit, col, lead, lag
from operator import add
from functools import reduce

def weighted_average(c, window, offsets, weights):
    assert len(weights) == len(offsets)

    def value(i):
        if i < 0: return lag(c, -i).over(window)
        if i > 0: return lead(c, i).over(window)
        return c

    values = [coalesce(value(i) * w, lit(0)) for i, w in zip(offsets, weights)]

    return reduce(add, values, lit(0))

In [73]:
w = Window.partitionBy("dummy_column").orderBy("DATEFORMAT")
# offsets, delays =  [-1,0], [0.5, 0.5]
a = [1/20] *20
offsets, delays =   [int(e) for e in list(np.arange(20)*-1)] , a
df = df.withColumn("Avg_Close_20", weighted_average(col("DAYOFMONTH"), w, offsets, delays))
df.show(2)

+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+
|               Date|    Open|    High|     Low|   Close|Adj_Close|   Volume|                 O-L|dummy_column|Next_Adj_Close|DATEFORMAT|YEAR|MONTH|DAYOFMONTH|DAYOFWEEK|WEEKOFYEAR|week|      Avg_Close_20|
+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+
|1980-12-12 00:00:00|0.513393|0.515625|0.513393|0.513393| 0.023007|117258400|                 0.0|     Stocks!|      0.024508|1980-12-12|1980|   12|        12|        6|        50|   2|0.6000000000000001|
|1980-12-15 00:00:00|0.488839|0.488839|0.486607|0.486607| 0.021807| 43971200|0.002232000000000...|     Stocks!|      0.025008|1980-12-15|1980|   12|        15|        2|        51|

In [74]:
a = [1/10] *10
offsets, delays =   [int(e) for e in list(np.arange(10)*-1)] , a
df = df.withColumn("Avg_Close_10", weighted_average(col("DAYOFMONTH"), w, offsets, delays))
df.show(2)

+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+
|               Date|    Open|    High|     Low|   Close|Adj_Close|   Volume|                 O-L|dummy_column|Next_Adj_Close|DATEFORMAT|YEAR|MONTH|DAYOFMONTH|DAYOFWEEK|WEEKOFYEAR|week|      Avg_Close_20|      Avg_Close_10|
+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+
|1980-12-12 00:00:00|0.513393|0.515625|0.513393|0.513393| 0.023007|117258400|                 0.0|     Stocks!|      0.024508|1980-12-12|1980|   12|        12|        6|        50|   2|0.6000000000000001|1.2000000000000002|
|1980-12-15 00:00:00|0.488839|0.488839|0.486607|0.486607| 0.021807| 43971200|0.002232000000000...|     S

In [75]:
a = [1/5] *5
offsets, delays =   [int(e) for e in list(np.arange(5)*-1)] , a
df = df.withColumn("Avg_Close_5", weighted_average(col("DAYOFMONTH"), w, offsets, delays))
df.show(2)

+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+------------------+
|               Date|    Open|    High|     Low|   Close|Adj_Close|   Volume|                 O-L|dummy_column|Next_Adj_Close|DATEFORMAT|YEAR|MONTH|DAYOFMONTH|DAYOFWEEK|WEEKOFYEAR|week|      Avg_Close_20|      Avg_Close_10|       Avg_Close_5|
+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+------------------+
|1980-12-12 00:00:00|0.513393|0.515625|0.513393|0.513393| 0.023007|117258400|                 0.0|     Stocks!|      0.024508|1980-12-12|1980|   12|        12|        6|        50|   2|0.6000000000000001|1.2000000000000002|2.4000000000000004|
|1980-12-15 00:00:00|0.48883

In [76]:
a = [1/80] *80
offsets, delays =   [int(e) for e in list(np.arange(80)*-1)] , a
df = df.withColumn("Avg_Close_80", weighted_average(col("DAYOFMONTH"), w, offsets, delays))
df.show(2)

+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+------------------+-------------------+
|               Date|    Open|    High|     Low|   Close|Adj_Close|   Volume|                 O-L|dummy_column|Next_Adj_Close|DATEFORMAT|YEAR|MONTH|DAYOFMONTH|DAYOFWEEK|WEEKOFYEAR|week|      Avg_Close_20|      Avg_Close_10|       Avg_Close_5|       Avg_Close_80|
+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+------------------+-------------------+
|1980-12-12 00:00:00|0.513393|0.515625|0.513393|0.513393| 0.023007|117258400|                 0.0|     Stocks!|      0.024508|1980-12-12|1980|   12|        12|        6|        50|   2|0.6000000000000001|1.20000

In [77]:
offsets, delays =   [0, 1, 8, 15], [0.25,0.25,0.25,0.25]
df = df.withColumn("Avg_Close_0_1_8_15", weighted_average(col("DAYOFMONTH"), w, offsets, delays))
df.show(2)

+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+------------------+-------------------+------------------+
|               Date|    Open|    High|     Low|   Close|Adj_Close|   Volume|                 O-L|dummy_column|Next_Adj_Close|DATEFORMAT|YEAR|MONTH|DAYOFMONTH|DAYOFWEEK|WEEKOFYEAR|week|      Avg_Close_20|      Avg_Close_10|       Avg_Close_5|       Avg_Close_80|Avg_Close_0_1_8_15|
+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+------------------+-------------------+------------------+
|1980-12-12 00:00:00|0.513393|0.515625|0.513393|0.513393| 0.023007|117258400|                 0.0|     Stocks!|      0.024508|1980-12-12|1980|   12|      

In [78]:
offsets, delays =   [0, 1, 3, 5], [0.25,0.25,0.25,0.25]
df = df.withColumn("Avg_Close_0_1_3_5", weighted_average(col("DAYOFMONTH"), w, offsets, delays))
df.show(2)

+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+------------------+-------------------+------------------+-----------------+
|               Date|    Open|    High|     Low|   Close|Adj_Close|   Volume|                 O-L|dummy_column|Next_Adj_Close|DATEFORMAT|YEAR|MONTH|DAYOFMONTH|DAYOFWEEK|WEEKOFYEAR|week|      Avg_Close_20|      Avg_Close_10|       Avg_Close_5|       Avg_Close_80|Avg_Close_0_1_8_15|Avg_Close_0_1_3_5|
+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+------------------+-------------------+------------------+-----------------+
|1980-12-12 00:00:00|0.513393|0.515625|0.513393|0.513393| 0.023007|117258400|                 0.0|  

In [79]:
offsets, delays =   [0, 1, 3, 5], [0.25,0.25,0.25,0.25]
df = df.withColumn("Avg_Close_0_1_3_5", weighted_average(col("DAYOFMONTH"), w, offsets, delays))
df.show(2)

+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+------------------+-------------------+------------------+-----------------+
|               Date|    Open|    High|     Low|   Close|Adj_Close|   Volume|                 O-L|dummy_column|Next_Adj_Close|DATEFORMAT|YEAR|MONTH|DAYOFMONTH|DAYOFWEEK|WEEKOFYEAR|week|      Avg_Close_20|      Avg_Close_10|       Avg_Close_5|       Avg_Close_80|Avg_Close_0_1_8_15|Avg_Close_0_1_3_5|
+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+------------------+-------------------+------------------+-----------------+
|1980-12-12 00:00:00|0.513393|0.515625|0.513393|0.513393| 0.023007|117258400|                 0.0|  

In [80]:
offsets, delays =   [0, 1, 5, 20, 80], [0.2,0.2,0.2,0.2,0.2]
df = df.withColumn("Avg_Close_0_1_5_20_80", weighted_average(col("DAYOFMONTH"), w, offsets, delays))
df.show(2)

+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+------------------+-------------------+------------------+-----------------+---------------------+
|               Date|    Open|    High|     Low|   Close|Adj_Close|   Volume|                 O-L|dummy_column|Next_Adj_Close|DATEFORMAT|YEAR|MONTH|DAYOFMONTH|DAYOFWEEK|WEEKOFYEAR|week|      Avg_Close_20|      Avg_Close_10|       Avg_Close_5|       Avg_Close_80|Avg_Close_0_1_8_15|Avg_Close_0_1_3_5|Avg_Close_0_1_5_20_80|
+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+------------------+-------------------+------------------+-----------------+---------------------+
|1980-12-12 00:00:00|0.513393|0.51

In [81]:
offsets, delays =   [0, 1, 2], [0.333,0.333,0.333]
df = df.withColumn("Avg_Close_3", weighted_average(col("DAYOFMONTH"), w, offsets, delays))
df.show(2)

+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+------------------+-------------------+------------------+-----------------+---------------------+------------------+
|               Date|    Open|    High|     Low|   Close|Adj_Close|   Volume|                 O-L|dummy_column|Next_Adj_Close|DATEFORMAT|YEAR|MONTH|DAYOFMONTH|DAYOFWEEK|WEEKOFYEAR|week|      Avg_Close_20|      Avg_Close_10|       Avg_Close_5|       Avg_Close_80|Avg_Close_0_1_8_15|Avg_Close_0_1_3_5|Avg_Close_0_1_5_20_80|       Avg_Close_3|
+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+------------------+-------------------+------------------+-----------------+-------------------

In [82]:
offsets, delays =   [0, 1], [0.5,0.5]
df = df.withColumn("Avg_Close_2", weighted_average(col("DAYOFMONTH"), w, offsets, delays))
df.show(2)

+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+------------------+-------------------+------------------+-----------------+---------------------+------------------+-----------+
|               Date|    Open|    High|     Low|   Close|Adj_Close|   Volume|                 O-L|dummy_column|Next_Adj_Close|DATEFORMAT|YEAR|MONTH|DAYOFMONTH|DAYOFWEEK|WEEKOFYEAR|week|      Avg_Close_20|      Avg_Close_10|       Avg_Close_5|       Avg_Close_80|Avg_Close_0_1_8_15|Avg_Close_0_1_3_5|Avg_Close_0_1_5_20_80|       Avg_Close_3|Avg_Close_2|
+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+------------------+-------------------+------------------+-------------

In [86]:
# drop nan rows - for removing nan rows created after lag features
df = df.na.drop()
df.show(2)

+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+------------------+-------------------+------------------+-----------------+---------------------+------------------+-----------+--------+--------------------+
|               Date|    Open|    High|     Low|   Close|Adj_Close|   Volume|                 O-L|dummy_column|Next_Adj_Close|DATEFORMAT|YEAR|MONTH|DAYOFMONTH|DAYOFWEEK|WEEKOFYEAR|week|      Avg_Close_20|      Avg_Close_10|       Avg_Close_5|       Avg_Close_80|Avg_Close_0_1_8_15|Avg_Close_0_1_3_5|Avg_Close_0_1_5_20_80|       Avg_Close_3|Avg_Close_2|     O-C|                 H-L|
+-------------------+--------+--------+--------+--------+---------+---------+--------------------+------------+--------------+----------+----+-----+----------+---------+----------+----+------------------+------------------+-----------

In [90]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj_Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- O-L: double (nullable = true)
 |-- dummy_column: string (nullable = false)
 |-- Next_Adj_Close: string (nullable = true)
 |-- DATEFORMAT: date (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAYOFMONTH: integer (nullable = true)
 |-- DAYOFWEEK: integer (nullable = true)
 |-- WEEKOFYEAR: integer (nullable = true)
 |-- week: string (nullable = true)
 |-- Avg_Close_20: double (nullable = false)
 |-- Avg_Close_10: double (nullable = false)
 |-- Avg_Close_5: double (nullable = false)
 |-- Avg_Close_80: double (nullable = false)
 |-- Avg_Close_0_1_8_15: double (nullable = false)
 |-- Avg_Close_0_1_3_5: double (nullable = false)
 |-- Avg_Close_0_1_5_20_80: double (nullable = f

#### Writing the processes data to csv

In [93]:
df.write.csv('AAPL_draft.csv')